<center> 
    <h1>LAB1 Vanilla GAN with MNIST dataset</h1>
</center>

* without any customization" 이라는 의미로 사용, 또는 "deault", "ordinary", "basic" 라고 이해

<h2> Lab 개 요 </h2>

* GAN 개념과 구조 이해  

* MNIST dataset을 이용한 vanilla GAN 구현  
  . MNIST : 0 ~ 9 숫자를 여러 스타일의 손글씨로 쓴 것  

* GAN 으로 FAKE 손글씨 숫자 생성(거의 똑 같은 수준 목표) 

<h3 style='color:green'> GAN 구조 </h3>
<img src="../Figures/GAN.png" width=700px>

<h3 style='color:green'> 중점학습내용</h3>

* Dataset 준비  
* 필요 Libs Import  
* Hyperparameters 설정  
* Generator and Discriminator  
* 시각화(Visualization)
* Training  
    
GAN 이해 참고자료(https://velog.io/@tobigs-gm1/basicofgan)

<h2>Dataset 준비</h2>

mnist dataset을 다운로드 받기위해 tensorflow.keras.datasets 에서 mnist 을 import

다음 코드 실행

In [4]:
from  tensorflow.keras.datasets  import  mnist

C:\Users\Owner\miniconda3\envs\jeff_heaton_t81_park\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Owner\miniconda3\envs\jeff_heaton_t81_park\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Owner\miniconda3\envs\jeff_heaton_t81_park\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\U

In [5]:
(x_train, _), (_, _) = mnist.load_data()   # 0 - 255  x_train에 load data

In [6]:
x_train = x_train/127.5 - 1                # 평균값(0 - 2) - 1 = -1 ~ 1 normalize

In [7]:
x_train.min(), x_train.max()

(-1.0, 1.0)

In [8]:
x_train = x_train.reshape(-1, 784)     # 28 x 28  사이즈 이미지를  1-Dim Vec 로 flatten => 784
x_train.shape

(60000, 784)

<h2>필요 모듈 Import</h2>


In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LeakyReLU, Dropout, Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
import numpy as np
import matplotlib.pyplot as plt

<h2>Hyperparameters 설정</h2>


In [10]:
NOISE_DIM = 10                      # gan input noise dimension  
adam = Adam(lr=0.0002, beta_1=0.5)  # adam optimizer, learning_rate, beta_1 값 설정 (fast convergence)

<h2> Generator</h2>


In [11]:
generator = Sequential([
    Dense(256, input_dim=NOISE_DIM), 
    LeakyReLU(0.2), 
    Dense(512), 
    LeakyReLU(0.2), 
    Dense(1024), 
    LeakyReLU(0.2), 
    Dense(28*28, activation='tanh'),
])

In [12]:
generator.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               2816      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              525312    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 784)               8

<h2>Discriminator</h2>

In [13]:
discriminator = Sequential([
    Dense(1024, input_shape=(784,), kernel_initializer=RandomNormal(stddev=0.02)),
    LeakyReLU(0.2), 
    Dropout(0.3), 
    Dense(512),
    LeakyReLU(0.2), 
    Dropout(0.3), 
    Dense(256),
    LeakyReLU(0.2), 
    Dropout(0.3), 
    Dense(1, activation='sigmoid')
])

In [14]:
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 1024)              803840    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)              

In [15]:
discriminator.compile(loss='binary_crossentropy', optimizer=adam) # discriminator 훈련을 위한 loss function / optimizer 

In [16]:
discriminator.trainable = False       # generator 훈련시 discriminator는 훈련하지 않음(고정, fixed)
gan_input = Input(shape=(NOISE_DIM,))
x = generator(inputs=gan_input)       # generator output => x : discriminator input
output = discriminator(x)

In [17]:
gan = Model(gan_input, output)        # gan model 구성

In [18]:
gan.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
sequential (Sequential)      (None, 784)               1463312   
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 1460225   
Total params: 2,923,537
Trainable params: 1,463,312
Non-trainable params: 1,460,225
_________________________________________________________________


In [19]:
gan.compile(loss='binary_crossentropy', optimizer=adam)  # generator 훈련을 위한 loss function / optimizer 

In [20]:
def get_batches(data, batch_size):   # 이미지 batch
    batches = []
    for i in range(int(data.shape[0] // batch_size)):
        batch = data[i * batch_size: (i + 1) * batch_size]
        batches.append(batch)
    return np.asarray(batches)

<h2>시각화(Visualization)</h2>

생성 이미지와 loss function 시각화를 위한 function 

In [21]:
def visualize_training(epoch, d_losses, g_losses):
    plt.figure(figsize=(8, 4))
    plt.plot(d_losses, label='Discriminator Loss')
    plt.plot(g_losses, label='Generatror Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    
    print('epoch: {}, Discriminator Loss: {}, Generator Loss: {}'.format(epoch, np.asarray(d_losses).mean(), np.asarray(g_losses).mean()))
    
    #Visualize after creating sample data
    noise = np.random.normal(0, 1, size=(24, NOISE_DIM))
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(-1, 28, 28)
    
    plt.figure(figsize=(8, 4))
    for i in range(generated_images.shape[0]):
        plt.subplot(4, 6, i+1)
        plt.imshow(generated_images[i], interpolation='nearest', cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.show()

<h2> Training </h2>

<img src="../Figures/d_g_loss.PNG" width=600 height=600 />

In [1]:
BATCH_SIZE = 128
EPOCHS= 50

In [ ]:
# This is a list for measuring loss of discriminator and gan model.
d_losses = []
g_losses = []

for epoch in range(1, EPOCHS + 1):
    # train for each batch
    for real_images in get_batches(x_train, BATCH_SIZE):
        # Generate random noise
        input_noise = np.random.uniform(-1, 1, size=[BATCH_SIZE, NOISE_DIM])
        
        # Generate fake image data
        generated_images = generator.predict(input_noise)
        
        # Define X data to learn on Gan
        x_dis = np.concatenate([real_images, generated_images])
        
        # Define Y data to be trained
        y_dis = np.zeros(2 * BATCH_SIZE)
        y_dis[:BATCH_SIZE] = 0.9
        
        # Discriminator training
        discriminator.trainable = True
        d_loss = discriminator.train_on_batch(x_dis, y_dis)
        
        # Gan training
        noise = np.random.uniform(-1, 1, size=[BATCH_SIZE, NOISE_DIM])
        y_gan = np.ones(BATCH_SIZE)
        
        # Train Generator
        discriminator.trainable = False
        g_loss = gan.train_on_batch(noise, y_gan)
        
    d_losses.append(d_loss)
    g_losses.append(g_loss)

    if epoch == 1 or epoch % 5 == 0:
        visualize_training(epoch, d_losses, g_losses)

<img src="result/result.png" width=500px>


### Lab1 Vanilla GAN with MNIST_실습종료 